In [ ]:
# run against production 2025-03-12
from csae_pyutils import gsheet_to_df
from django.core.exceptions import ObjectDoesNotExist

from tqdm import tqdm
import pandas as pd
from normdata.utils import import_from_normdata

In [ ]:
df = gsheet_to_df("1Txis3KRl9iO2Ft3Y0HbZL7WZ6ZrBQrd2jwfDNAqgQKA")
len(df)

In [ ]:
col, _ = Collection.objects.get_or_create(name="Der Brenner")
domain = "brenner"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = "https://brenner.oeaw.ac.at/"
col.collection_type = col_type
col.save()

In [ ]:
# broken_gnd = []
# pmb_uris = []
# for i, row in tqdm(df.iterrows(), total=len(df)):
#     if isinstance(row["gnd"], str):
#         gnd_uri = row["gnd"].replace("https://reconcile.gnd.network/gnd", "https://d-nb.info/gnd")
#         domain_uri = f"https://brenner.oeaw.ac.at/?p=brenner{row['code']}".replace("_", "")
#         entity = import_from_normdata(gnd_uri, "person")
#         if entity:
#             pmb_uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
#             pmb_uri.entity = entity
#             pmb_uri.save()
#             entity.collection.add(col)
#             pmb_uris.append([row['code'], entity.id])
#         else:
#             broken_gnd.append(gnd_uri)


In [ ]:
# process only real perons without gnd
df = df[df['gnd'].isna()]
df = df[df['nameType'] == "2"]
len(df)

In [ ]:
born_in = PersonPlaceRelation.objects.get(id=88)
died_in = PersonPlaceRelation.objects.get(id=89)
for i, row in tqdm(df.iterrows()):
    domain_uri = f"https://brenner.oeaw.ac.at/?p=brenner{row['code']}".replace("_", "")
    try:
        uri = Uri.objects.get(uri=domain_uri)
        entity = uri.entity
        entity = Person.objects.get(id=entity.id)
        if pd.notna(row["deathDate"]):
            entity.end_date_written = row["deathDate"]
    except ObjectDoesNotExist:
        if ', ' in row["name"]:
            name, first_name = row["name"].split(", ")[0:2]
        else:
            name = row["name"]
            first_name = None
        item = {
            "name": name,
            "first_name": first_name,
        }
        if pd.notna(row["birthDate"]):
            item["start_date_written"] = row["birthDate"]
        if pd.notna(row["deathDate"]):
            item["end_date_written"] = row["birthDate"]
        entity = Person.objects.create(**item)
        entity.collection.add(col)
        uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
        uri.entity = entity
        uri.save()
    if row["sex"] == "m":
        entity.gender = "male"
        entity.save()
    if row["sex"] == "f":
        entity.gender = "female"
        entity.save()
    if pd.notna(row["birthPlace"]):
        try:
            place, _ = Place.objects.get_or_create(name=row["birthPlace"])
        except:
            place = Place.objects.filter(name=row["birthPlace"]).reverse()[0]
        if pd.notna(row["birthDate"]):
            start_date = row["birthDate"]
        else:
            start_date = None
        pp, _ = PersonPlace.objects.get_or_create(
            related_person=entity,
            related_place=place,
            relation_type=born_in,
            start_date_written=start_date,
            end_date_written=start_date
        )
        
    if pd.notna(row["deathPlace"]):
        try:
            place, _ = Place.objects.get_or_create(name=row["deathPlace"])
        except:
            place = Place.objects.filter(name=row["deathPlace"]).reverse()[0]
        if pd.notna(row["deathDate"]):
            start_date = row["deathDate"]
        else:
            start_date = None
        
        pp, _ = PersonPlace.objects.get_or_create(
            related_person=entity,
            related_place=place,
            relation_type=died_in,
            start_date_written=start_date,
            end_date_written=start_date
        )
    if pd.notna(row["profs"]):
        profs = []
        for x in row["profs"].split(", "):
            prof = ProfessionType.objects.filter(name__startswith=x)
            if prof:
                for y in prof:
                    profs.append(y)
            else:
                prof, _ = ProfessionType.objects.get_or_create(name=x)
                profs.append(prof)
        if profs:
            entity.profession.add(*profs)
            